### SQL - Parte 2
---

> Data: 06/10/2021<br>
> Aula: 02 <br>
> Módulo: 01 <br>

#### Objeto de estudo:
---

Usaremos dados de exemplo do site [MySQL Tutorial](http://www.mysqltutorial.org/mysql-sample-database.aspx).
O SQLite é bem mais simples e didático e opera num arquivo local.

#### Visualizando o banco de dados, suas tabelas e dados:
---

![database example](http://www.mysqltutorial.org/wp-content/uploads/2009/12/MySQL-Sample-Database-Schema.png)

Cada SGBD tem seu próprio ferramental para navegar nos dados. MariaDB/MySQL tem o popular [PHP MyAdmin](https://www.phpmyadmin.net), Oracle e DB2 tem suas próprias ferramentas proprietárias, SQLite tem [DB Browser for SQLite](https://sqlitebrowser.org) e uma ferramenta online chamada [SQLite Online](https://sqliteonline.com).

#### Para navegar nos dados do DB SQLite que acabamos de criar, use uma das 3 opções:
---

* Use o comando `sqlite3`.
* Instale o **DB Browser for SQLite** e abra o arquivo.
* Envie o arquivo para o site **SQLiteOnline.com**.

##### 1 - Configuração do notebook:
---

In [1]:
# Bibliotecas:
import pandas as pd
import sqlite3

A operação mais comum em um banco de dados é a leitura de dados, e para isso precisamos escrever um requerimento, que é mais conhecido como QUERY.

Uma query poder ser quebrada em partes:
1. SELECT = irá iniciar a seleção de dados que faremos a leitura
2. FROM = irá selecionar as tabelas de interesse
3. WHERE = vamos declarar as condições para ler os dados
4. GROUP BY = podemos agrupar algum dado considerando uma coluna 
5. Pós processamento = podemos ordenar (ORDER BY) ou estabelecer limites (LIMIT)

In [2]:
# Conexão ao banco de dados:
sqliteConnection = sqlite3.connect('database1.db')
cursor = sqliteConnection.cursor()
print("Conectado ao banco de dados!")

Conectado ao banco de dados!


##### 2 - Verificação das tabelas do banco de dados:
---

In [3]:
res = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print(name[0])

my_table
customers
employees
offices
orderdetails
orders
payments
productlines
products
countries


In [4]:
## vamos ver a tabelas customers:
query = """SELECT * FROM  customers;""" 
df_query = pd.read_sql_query(query,sqliteConnection)
df_query.head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370.0,118200
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504.0,81700


In [5]:
## vamos ler apenas colunas específicas:
query = """SELECT customerName, city, country FROM customers; """
df_query = pd.read_sql_query(query,sqliteConnection)
df_query.head()

,customerName,city,country
0,Atelier graphique,Nantes,France
1,Signal Gift Stores,Las Vegas,USA
2,"Australian Collectors, Co.",Melbourne,Australia
3,La Rochelle Gifts,Nantes,France
4,Baane Mini Imports,Stavern,Norway


#### Verificar o valor total gasto em cada fornecedor:
---

- quantidade pedida * valor pedido (priceEach * quantityOrdered)

In [6]:
query = """select * from products as p; """
df_query = pd.read_sql_query(query,sqliteConnection)
df_query.head()

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00


In [7]:
query = """select * from orderdetails as o;"""
df_query = pd.read_sql_query(query,sqliteConnection)
df_query.head()

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4
3,10100,S24_3969,49,35.29,1
4,10101,S18_2325,25,108.06,4


In [8]:
# Group by
# Se for usar o Order by junto com o group by, lembrar de colocar o order by após o group by
query = """ select 
            p.productVendor as estabelecimento,
            sum(p.buyPrice * o.quantityOrdered) as 'total'
            
            from products as p            
            inner join orderdetails as o
            on p.productCode = o.productCode           
            
            group by
            1
            
            order by
            total desc;

        """
df_query = pd.read_sql_query(query,sqliteConnection)
df_query

,estabelecimento,total
0,Classic Metal Creations,550116.36
1,Exoto Designs,510855.22
2,Motor City Art Classics,499628.70
3,Unimax Art Galleries,491829.94
4,Gearbox Collectibles,489790.15
5,Welly Diecast Productions,477268.23
6,Second Gear Diecast,465824.99
7,Min Lin Diecast,448234.32
8,Highway 66 Mini Classics,418761.04
9,Carousel DieCast Legends,415487.66


#### COUNT
#### Contar quantidade de produtos  adquiridos  em cada fornecedor(productVendor):

In [9]:
query = """select * from products as p; """
df_query = pd.read_sql_query(query,sqliteConnection)
df_query

,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles,1:10,Min Lin Diecast,"This replica features working kickstand, front...",7933,48.81,95.70
1,S10_1949,1952 Alpine Renault 1300,Classic Cars,1:10,Classic Metal Creations,Turnable front wheels; steering function; deta...,7305,98.58,214.30
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles,1:10,Highway 66 Mini Classics,"Official Moto Guzzi logos and insignias, saddl...",6625,68.99,118.94
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles,1:10,Red Start Diecast,"Model features, official Harley Davidson logos...",5582,91.02,193.66
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars,1:10,Motor City Art Classics,Features include: Turnable front wheels; steer...,3252,85.68,136.00
...,...,...,...,...,...,...,...,...,...
105,S700_3505,The Titanic,Ships,1:700,Carousel DieCast Legends,"Completed model measures 19 1/2 inches long, 9...",1956,51.09,100.17
106,S700_3962,The Queen Mary,Ships,1:700,Welly Diecast Productions,Exact replica. Wood and Metal. Many extras inc...,5088,53.63,99.31
107,S700_4002,American Airlines: MD-11S,Planes,1:700,Second Gear Diecast,Polished finish. Exact replia with official lo...,8820,36.27,74.03
108,S72_1253,Boeing X-32A JSF,Planes,1:72,Motor City Art Classics,"10\"" Wingspan with retractable landing gears.C...",4857,32.77,49.66


In [10]:
query = """select p.productVendor, count(*) as qtd from products as p group by 1; """
df_query = pd.read_sql_query(query,sqliteConnection)
df_query

,productVendor,qtd
0,Autoart Studio Design,8
1,Carousel DieCast Legends,9
2,Classic Metal Creations,10
3,Exoto Designs,9
4,Gearbox Collectibles,9
5,Highway 66 Mini Classics,9
6,Min Lin Diecast,8
7,Motor City Art Classics,9
8,Red Start Diecast,7
9,Second Gear Diecast,8


#### Join
#### Vamos verificar os consumidores japoseses no ano de 2004

In [11]:
query = '''
SELECT a.contactFirstName,
       a.state,
       a.country,
       b.paymentDate,
       sum(b.amount) as amount
FROM customers a
LEFT JOIN payments b
on (a.customerNumber = b.customerNumber)
WHERE a.country = 'Japan'
and b.paymentDate >= DATE('2004-01-01')
and b.paymentDate <= DATE('2004-12-31')
GROUP by 1,2,3,4
ORDER BY 4 ASC;
'''
df_query = pd.read_sql_query(query,sqliteConnection)
df_query

,contactFirstName,state,country,paymentDate,amount
0,Mory,Osaka,Japan,2004-01-19,47177.59
1,Mory,Osaka,Japan,2004-04-17,15183.63
2,Akiko,Tokyo,Japan,2004-06-21,22037.91
3,Akiko,Tokyo,Japan,2004-11-29,48927.64


In [12]:
query = '''
SELECT a.contactFirstName,
       a.state,
       a.country,
       b.paymentDate,
       sum(b.amount) as amount
FROM customers a
LEFT JOIN payments b
on (a.customerNumber = b.customerNumber)
WHERE a.country = 'UK'
and b.paymentDate between '2003-01-01' and '2003-12-01'
GROUP by 1,2,3,4
ORDER BY 4 ASC;
'''
df_query = pd.read_sql_query(query,sqliteConnection)
df_query

,contactFirstName,state,country,paymentDate,amount
0,Rachel,None,UK,2003-03-27,48425.69
1,Ann,None,UK,2003-07-07,29429.14
2,Elizabeth,None,UK,2003-10-20,23908.24
3,Ann,None,UK,2003-11-23,37455.77
